In [4]:
using Pkg
Pkg.activate(".")
using OptimalTransport
using LinearAlgebra
using SparseArrays
using Distances
using StatsBase
using NNlib
using BenchmarkTools
using NearestNeighbors
using RandomMatrix
using ParallelNeighbors
using ProgressMeter

include("../src/util.jl")

  Activating project at `/data/gpfs/projects/punim0638/stephenz/qrot/notebooks`


norm_kernel (generic function with 1 method)

In [5]:
function gen_data(N, d)
    μ_spt = randn(N, d)
    X = μ_spt
    μ = ones(N)
    C = sum(X.^2 ; dims = 2)/2 .+ sum(X.^2 ; dims = 2)'/2 - X * X'
    Cmean = mean(C)
    C[diagind(C)] .= Inf
    return X, μ, C / Cmean
end

function knn_adj_parallel(X, k)
    indices, _ = ParallelNeighbors.knn(X, X, k);
    A = zeros(size(X, 2), size(X, 2));
    @inbounds for i = 1:size(A, 1)
        A[i, i] = 1; @inbounds for j in indices[i] A[i, j] = 1 end
    end
    A = sparse(A)
end
function construct_support(X, k)
    # S = knn_adj(X', k);
    S = knn_adj_parallel(X', k);
    S += sign.(mean([randPermutation(size(X, 1)) for _ in 1:k]));
    S += S';
    S = sign.(S); 
    S[diagind(S)] .= 0;
    dropzeros!(S);
    return S
end

construct_support (generic function with 1 method)

In [6]:
ε = 1.0
k = 50
N = 10_000
d = 100

100

In [ ]:
X, μ, C = gen_data(N, d);
ENV["JULIA_DEBUG"] = ""; 
elapsed_dense = @showprogress [@elapsed sparse(quadreg(μ, C, ε, OptimalTransport.SymmetricQuadraticOTNewton(); maxiter = 100)) for _ in 1:10]
elapsed_sparse = @showprogress [@elapsed quadreg(μ, C, ε, OptimalTransport.SymmetricQuadraticOTNewtonAS(construct_support(X, k)); maxiter = 100) for _ in 1:10]

In [8]:
elapsed_dense

10-element Vector{Float64}:
 79.08055256
 77.956411602
 76.198541543
 76.7541346
 76.249853094
 79.836221955
 77.728582493
 76.562115567
 79.325001951
 88.395609122

In [9]:
elapsed_sparse

10-element Vector{Float64}:
 18.609266564
 13.296809725
 15.010748704
 14.588107647
 15.797307508
 15.644214446
 14.231254935
 17.445514924
 12.918317531
 12.912251602